In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("rdd_api").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/02 21:34:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext

## List of Distinct Genres

In [26]:
def save_all_genres():
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    genres_rdd = movies_rdd.map(lambda x: x.split("::")[2])
    genres_rdd = genres_rdd.flatMap(lambda x: x.split("|"))
    distinct_genres_rdd = genres_rdd.distinct().sortBy(lambda x: x)
    distinct_genres_rdd.saveAsTextFile("processed_data/distinct_genres")

In [12]:
save_all_genres()

/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")
/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")


In [14]:
def get_all_genres():
    data = sc.textFile("processed_data/distinct_genres").collect()
    return data

In [15]:
print(get_all_genres())

['Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir']


In [27]:
def save_list_of_years():
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    title_year_rdd = movies_rdd.map(lambda x: x.split("::")[1])
    year_rdd = title_year_rdd.map(lambda x: x[-5:-1])
    distinct_years_rdd = year_rdd.distinct().sortBy(lambda x: x)
    distinct_years_rdd.saveAsTextFile("processed_data/list_of_years")

In [19]:
save_list_of_years()

/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")
/Users/ankitrajvanshi/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
  warnings.warn("Please install psutil to have better " "support with spilling")


In [28]:
def save_list_of_starting_letters():
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    title_year_rdd = movies_rdd.map(lambda x: x.split("::")[1])
    first_char_rdd = title_year_rdd.map(lambda x: x[0])
    distinct_first_char_rdd = first_char_rdd.distinct().sortBy(lambda x: x)
    distinct_first_char_rdd.saveAsTextFile("processed_data/list_of_starting_letters")

In [24]:
save_list_of_starting_letters()

In [24]:
def top_n_most_viewed_movies(num):
    ratings_rdd = spark.sparkContext.textFile('Dataset/ratings.dat')
    movies = ratings_rdd.map(lambda x: x.split("::")[1])
    movies = movies.map(lambda x: (x,1))
    movies = movies.reduceByKey(lambda x,y: x + y)
    top_num_movies = sc.parallelize(movies.sortBy(lambda x: x[1]).take(num))
    
    movies_rdd = spark.sparkContext.textFile('Dataset/movies.dat')
    title_rdd = movies_rdd.map(lambda x: (x.split("::")[0], x.split("::")[1][:-7]))
    top_num_movies_title = title_rdd.join(top_num_movies).sortBy(lambda x: x[1]).map(lambda x: (x[0], x[1][0]))
    
    top_num_movies_title.saveAsTextFile("processed_data/top_n_most_viewed_movies")
    

In [25]:
top_n_most_viewed_movies(10)

In [38]:
def get_top_n_most_viewed_movies_by_gender(num, gender):
    
    try:
        data = sc.textFile("processed_data/top_{}_most_viewed_movies_by_{}".format(num,gender)).collect()
    except Exception as e:
#         print(e)
        error = "top_{}_most_viewed_movies_by_{} File Not Found".format(num, gender)
        return error
    
    return data

In [39]:
get_top_n_most_viewed_movies_by_gender(9, "M")

'top_9_most_viewed_movies_by_M File Not Found'

In [40]:
def get_top_n_most_liked_movies_by_gender(num, gender):
    
    try:
        data = sc.textFile("processed_data/top_{}_most_liked_movies_by_{}".format(num,gender)).collect()
    except Exception as e:
#         print(e)
        error = "top_{}_most_viewed_movies_by_{} File Not Found".format(num, gender)
        return error

    return data

In [42]:
get_top_n_most_liked_movies_by_gender(10, "F")

['Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode IV - A New Hope',
 'Princess Bride, The',
 'E.T. the Extra-Terrestrial',
 'L.A. Confidential',
 'Wizard of Oz, The',
 'Fargo',
 'Men in Black',
 'American Beauty']